In [1]:
import pandas as pd
from pandas import option_context
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import MWETokenizer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer 

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

from collections import Counter

from textblob import TextBlob

In [2]:
df = pd.read_csv('04-data/preprocessed_app_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45536 entries, 0 to 45535
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     45536 non-null  object 
 1   description               45536 non-null  object 
 2   summary                   45520 non-null  object 
 3   installs                  45536 non-null  object 
 4   minInstalls               45536 non-null  float64
 5   score                     45536 non-null  float64
 6   ratings                   45536 non-null  float64
 7   reviews                   45536 non-null  float64
 8   histogram                 45536 non-null  object 
 9   price                     45536 non-null  float64
 10  free                      45536 non-null  int64  
 11  currency                  45536 non-null  object 
 12  sale                      45536 non-null  bool   
 13  offersIAP                 45536 non-null  bool   
 14  inAppP

In [9]:
df = df[df['description_clean'].notna()]

In [10]:
df_analysis = df[['title','description_clean']].copy()
df_analysis.reset_index(drop=True,inplace=True)

In [11]:
df_analysis

,title,description_clean
0,World War 2: Offline Strategy,command allies in 25 epic world war 2 locatio...
1,AndroXLS editor for XLS sheets,androxls is an android app to edit xls spreads...
2,SoundSeeder -Play music simultaneously and in ...,soundseeder syncs up your music playback on mu...
3,LibreOffice & OpenOffice document reader | ODF,view and modify documents created using libreo...
4,"All PDF - PDF Reader, PDF Viewer & PDF Converter",pdf reader for android pdfs converter free is ...
...,...,...
45526,Bikini Bottom Maps for Minecraft,play the best maps of bikini bottom with this ...
45527,PokeCraft : Monsters Mod,the pokecraft monsters mod for minecraft is a...
45528,TongitsXtreme,enjoy this exciting filipino card game called ...
45529,Play Tube,---------- play tube music ---------this app d...


In [12]:
stop = stopwords.words('english')

In [25]:
vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_analysis.description_clean)

In [28]:
nmf_model = NMF(12)
doc_topic = nmf_model.fit_transform(doc_word)
topic_word = nmf_model.components_

In [29]:
words = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words = [[words[e] for e in l] for l in t]
topic_words

[['game', 'play', 'free', 'new', 'world', 'fun'],
 ['app', 'free', 'use', 'phone', 'get', 'time'],
 ['photo', 'editor', 'photos', 'frames', 'effects', 'make'],
 ['car', 'racing', 'driving', 'parking', 'stunt', 'game'],
 ['video', 'videos', 'maker', 'status', 'download', 'app'],
 ['music', 'player', 'free', 'songs', 'play', 'audio'],
 ['keyboard', 'theme', 'themes', 'new', 'typing', 'hd'],
 ['truck', 'driving', 'bus', 'simulator', 'transport', 'city'],
 ['robot', 'games', 'flying', 'transform', 'transforming', 'car'],
 ['de', 'la', 'en', 'para', 'que', 'el'],
 ['games', 'shooting', 'free', 'best', 'new', 'kids'],
 ['wallpaper', 'live', 'wallpapers', 'screen', 'phone', 'hd']]